In [1]:
%%configure -f
{
 "executorCores": 4,
 "executorMemory": "47696M",
 "conf": {"spark.default.parallelism": 1000,
          "spark.sql.shuffle.partitions": 1000,
          "spark.task.cpus": 1,
          "spark.kryoserializer.buffer.max": "2047m",
          "spark.driver.maxResultSize": "10g"
         }
}

In [2]:
import math
import os
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import pyspark.sql.types as T
from collections import defaultdict

import smart_open

from scipy.sparse import csr_matrix
import scipy

import numpy as np

from itertools import islice

import pickle

import graphene

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1574845575997_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# spark = (
#     SparkSession.builder
#      .master("local[*]")
#      .appName("SparkTest")
#      .config("spark.driver.memory", "10g")
#      .getOrCreate()
#      )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
word2id_path = "s3://oaknorth-ml-dev-eu-west-1/company2vec/common/bow_description_lemmatized"
data_path = "s3://oaknorth-ml-dev-eu-west-1/company2vec/data_desc_only/raw_company_features_bow_description_lemmatized"
model_path = "s3://oaknorth-ml-dev-eu-west-1/company2vec/model/probabilistic_lemmatized/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
companies = spark.read.load("s3://ai-data-lake-dev-eu-west-1/business/company_data_denormalized")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
with smart_open.open(f"{model_path}/simple_LDA/parameters/phi.np", "rb") as f:
    phi = np.load(f, allow_pickle=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
industry2idx = {}
with smart_open.open(f"{model_path}/simple_LDA/industries", "r") as f:
    for line in f:
        toks = line.strip().split(",")
        industry,idd = ",".join(toks[:-1]), toks[-1]
        industry2idx[industry] = int(idd)
industries = list(industry2idx.keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
word2id = {}
with smart_open.open(f"{word2id_path}/word2id.csv", "r") as f:
    for line in f:
        word,idd = line.strip().split(",")
        word2id[word] = int(idd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
len(word2id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

41164

In [9]:
phi.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1322, 41164)

In [10]:
id2word = {}
for word,idd in word2id.items():
    id2word[idd] = word

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
public_labels = {
'1b3872ba-8913-517f-a253-7e62a525cf55':['f702fd12-611d-51ba-b61d-c95d505516f5', 'e449976b-580d-546a-aeb2-475a36c48d5c', '91a4576f-d032-5e06-a8a9-5753e836a2d1', '2978b28a-96d5-5c2e-b212-39893c36ed0a'],
'7689e398-0970-5c18-ba93-e0b14013ab30':['e5e55e0c-98f2-57e6-b252-2acd32197a33', 'dbd74ef8-7e63-587c-a0db-02846764cebd', '29f2b914-f655-5a0e-8642-252840119f94'],
'cb997a23-552a-52b9-acf5-bcdce2d1eea1':['f216296d-2161-54dc-a567-72a3043bf55b', '7ee2b159-1c7b-5867-92dd-8323be244577', 'b21e9a87-c5a6-5906-9d60-518f12f4a015'],
'e62877cd-15cd-561d-a77e-8a5bccf9873c':['ea39af2b-a76b-5df5-bcd1-c785cabedfe0', '9f8a1e48-a744-50cb-9b0d-87febeddb95b'],
'29f2b914-f655-5a0e-8642-252840119f94':['ea39af2b-a76b-5df5-bcd1-c785cabedfe0', '44b37e05-b67e-5ab4-bc3c-6d9e090ac13d', '07e02755-93b9-51d4-ab1f-67bc3ac4593c'],
'c655ffcb-391e-57af-9d57-28586381be92':['74116d9c-f19e-5089-b8d0-894befb4309b', 'bc68fb91-85c2-5650-bca5-dfbd0a1dd1ee', '7689e398-0970-5c18-ba93-e0b14013ab30'],
'b01d22f1-2b08-5a1b-937e-bd29be833832':['6d4e8b59-6790-5bad-9578-dfc5e23d40e6', 'a3c20fd1-df51-5c6b-9cbc-f1e5dd2ac129', 'aae8652a-0670-5643-978b-8e596552b4ff', '5147c270-2d82-5f2f-9081-62966a777e63', '7d48041c-b5b4-531b-9570-7deff9cc6c2e', 'a22a6d38-8932-5bb2-859b-cbb8ad9cf2a9', 'bffaf4bf-d404-5af9-bd7a-08110d90d39e', '8c0231ed-d771-5821-933e-15a0be075d88', 'a659d2df-a86d-595a-8420-1d42f82d66e4', 'd96d1bd4-fa15-5267-a659-42f973d5603b'],
'6a99e295-db08-5598-9746-10848bd2871e':['fc7226d1-910a-5010-bdb9-2d7865355965', 'd3189f4b-7dd2-5751-a8d2-a2121b0b4790', '7f943915-83be-505f-b065-bd45ccba65e2'],
'eb6a1519-3559-5797-ac35-d31505d66961':['6deb9c74-e7ed-551a-b88f-a107678b22c1', 'aa0ef5d1-7a0e-518d-b93d-1834db5bae48', 'ddc78420-8f1e-5656-a2fa-c967f278aad1', '65c843d5-e8b8-537f-a6c4-3ccb301d09e5', 'ba3be4f0-592d-5d72-a96b-5c461c871c9b', '522170f9-6489-52b8-bba6-ccb4ea4663ee', '005d3b86-14de-58a3-86ac-16ceff561782'],
'94a8555e-9f82-5657-9f32-f44c1844e371':['c7c22dd5-005b-56f1-8bd3-3420cc515895', '7fc2ed34-67db-5aed-bd9b-4a93d7981384', '8a586930-f14b-5414-902d-9c011a035e22', 'aab5d77c-76a1-5a64-ba9e-9db25d7126a0', 'f83f09b6-7a21-50c4-9cf9-730278fe565e'],
'4dfb3296-c5ab-5d27-bcaa-71a67990dc0f':['5db27606-2ba6-5599-aab1-754449cbf30f', '8b881676-4524-54fb-8d04-e9d80a70adc6'],
'3e71b765-5a7c-54b0-8687-fbb479e5ce91':['cc6a39ed-58d8-55d0-909f-ba1fba65159c', '42f7f27d-ed06-58e3-88c4-0d8e347991d7', '40cb3da1-7886-575f-b208-dd5e3d2e50ec', '648c94fc-5575-5e26-b284-66332d12d98d', 'db6e81fa-d08d-5597-bc1c-f3e97c72e3f3', 'c9fddd7f-7bd2-564a-acab-a776d09988cb'],
'4b90ea7b-dd01-5114-b2b1-e44f8b0f496c':['ff551276-08ea-5656-a96e-8f9b7c620de7', 'bade0809-2003-5dbb-bf6d-1aefe7347bd5', '2c437ad1-51ed-52bc-ac6f-b9d549a81370', '2cdc6f88-488e-5a6a-92e0-7d0c3a6f9c6b', 'a06b3a79-9328-5cc5-b8d1-77a773faa0bc'],
'0eac306f-7b21-534f-b7eb-2fe3dbf0dcb9':['a255ad13-8d70-5d14-9431-079d5167750e', '95a58411-5323-59d9-ba02-0e746f8ea16e', 'c2ad7853-588c-5dcf-ad06-a295173586eb', '15fd56d7-eaf3-572b-b11d-ea4a8c40dd7c', '8f96bdea-26b6-5304-a068-86bc974e0cff', '78b205cc-391a-5f9c-adfd-289a5027fead'],
'6d6a0abb-b4fa-5dfa-b44e-35927bff8864':['c474b779-c670-590f-b3e6-f3c48a87ff72', 'a4c75988-d2e2-5aef-ac41-20137a78de9d', '3d6393a2-0c9a-5886-b5ed-0dfe331bb9bc', '057fe70d-3f02-5132-a580-50314a6da51e', '91b185f6-5c35-55dd-9c43-1f5789f4693f', '448b7798-a147-5c15-8052-78843751d0ea'],
'3f02111a-3779-53cb-b556-ef2b02712eb8':['5df1139a-fd1f-5e47-ba24-ba6d46f29c98', '2a725ab6-c3a6-55f4-b3b5-61197a6fb7f4', '905898bd-b3a7-59e8-81b9-35bdbd682f5a', 'a265e9b0-1722-5826-ae35-dd64cecc8cd3', '63c9b56f-726f-539d-a347-cbcbcabc7f3a'],
'a2ec645e-4b24-5995-ac16-a91d2fe427d8':['6e59ae8b-a73e-5819-abde-b6275f9afd44', '35615ff3-9c28-5f4d-96d4-75ee7e861ab4', '9806a724-a62a-53f3-bba4-481f40fbecd6', '893f7929-8f1e-5472-a209-503b6fb1b028', 'fb7f54de-114b-56e9-9fc5-6c3011e104ab', '491918a7-5259-5a5e-b01f-4842a98ec78d', '0c04c0a9-fa85-548d-bd89-35300e16b28c', '16584e89-1b38-5bbb-ba45-a7bfc6dcc8e8'],
'c2ad7853-588c-5dcf-ad06-a295173586eb':['6819ac99-458c-5922-8861-14204f4ba663', '7e14c6f9-7e3f-5070-a550-d278ce7d77cc', '7d1e3f8e-a63c-551e-918d-94a09f2a94b1', '102771ca-59e9-5017-8061-54f4d6c79b3e', '01e5edf1-5304-55f7-84f7-570fe39199f8', '262e95ff-8274-5b39-b0f8-2e5c8d980220'],
'7a93383a-3f64-5d88-b0ca-f25210a26530':['19f0adde-24bb-59d3-a21f-a6583714f9a1', 'ab0a0b11-5984-55e5-9ef8-33b003577a0a', '93d033f8-4541-575b-96f8-fb3366b3eb77', 'ff9107a2-0017-57bc-b2aa-8e8880d11598', 'fff722a2-0908-5198-9a69-64612d28f92e', '0b314379-45ee-528d-804d-ef7478198af6', '21298d2b-ce8b-5aef-ba4b-25399928c53f', 'cf1fb9ed-b7a9-5aa0-acbd-69505ead7ce2', '0008f3ef-65a1-5852-91ca-e2de3c8cea7e', 'cfcd8e1d-ec5a-55ae-a82f-8e18f9c2e7ec'],
'8b881676-4524-54fb-8d04-e9d80a70adc6':['6cde2a53-9dc2-5790-b9d9-3e98d685f9b0', '03a57c6b-eef1-5407-a376-9dbc57722c20', '42fd29d1-ec85-561a-94d7-0b010dfba9f7', '4dfb3296-c5ab-5d27-bcaa-71a67990dc0f', '8822903e-754e-577d-9785-fed317439051', '29f2b914-f655-5a0e-8642-252840119f94', '5db27606-2ba6-5599-aab1-754449cbf30f'],
'5e6d242e-3a76-5145-9c3d-9cde3e69fd41':['7e8b428e-dee1-5a3e-a022-afd5940893b3', '91e45209-0c47-51de-b12a-5b8d516b7a2b'],
'44aba920-f4af-547e-84e6-68db5b56617a':['f599c68c-b3be-5beb-ae2c-68538805afe7', 'a196b19e-0313-5588-a132-1a920ff3c9cd', '0d002ee3-3822-5835-b49c-ae18bf616991', '97fdace6-f059-57b6-8e59-5d39e33364ca', '0c8790f8-e268-50c0-8861-e7103fda0db4'],
'1ed96474-1404-5a0e-b5b5-34f1c0638989':['6129066c-1a86-5960-8be5-1d79e52024cb', 'a674ad5e-2de2-522a-b99b-c21b9a815ccb', 'b1a34bd8-63b5-52c3-86c3-3c83409b96bf', '9155ea42-371d-5bc7-950b-5c93e151b4be', 'f5f4427a-f883-5ad7-acfe-adb68e84067c'],
'87bf6c8a-3ca4-5f9d-b552-8d9f857ebd74':['3cfe2a85-be8e-5a29-90ec-df7568441529', '7878a4dd-803d-597c-9773-038931bb341d', '67050e4c-7771-5a63-b48f-8f699f6fd53b', '5ced6ecb-26fd-50a8-b16e-f9844203b854', 'bf155ea4-adda-50ea-aec5-ff3bd4b493e4', '6cfe0483-aacf-56b6-baec-0a30757b4888', '88197a8b-8cc4-56b8-bac1-7c0e92826ce6', '606d25b9-82c9-5862-b5bb-6106b7d649ef', 'a920a145-b5a5-52e3-9137-5468466d87b7'],
'1c599f2e-4e28-5694-9647-0d06f5896a88':['05bb9448-713e-547d-b59c-8a4464e98702', '22c47d21-3704-58a1-bb7d-a3bf98cc766e', 'd9a65fea-a863-578d-81d4-577a9b4f7a03', '98ad153e-e53f-5337-b21a-ee16ebb15ce7'],
'2a07b0ed-7a52-5641-9e9a-e15e04bd2faa':['2500dc34-48b5-53eb-9d15-3ac2431fe1a3', 'c89aa997-e62c-53c5-a3ae-e657297d208a', 'c5b2ae28-843a-5701-872e-c182df1e64ce', 'f57b274a-1598-5e79-b8bf-db8690fd3f50', '363885b4-e937-529e-b9eb-b10989879494', 'f995b9fe-fb7c-5b0f-9673-30203b4e9c1a'],
'27a20afb-730c-5243-bae9-21de6ae9a95d':['fdcb3a4c-1db3-597c-b885-093937c8a68c', '6614e817-c2ae-56ba-8419-5965ab8600ed', '98ac98f5-d676-5eee-9602-c874f6d86f15', 'e52681c3-ac43-5d84-821e-0a8f2323f49f', '3de135e6-0263-5d65-97a9-0b1ffc473c6d'],
'4294552e-cc06-5002-a418-5c295e89f1d7':['7d128a0a-7a33-5ec7-994e-be6b5b44d58a', '78d39ee5-22a5-570e-9103-d918b64296e8', 'ea7463ff-afb9-5e57-b63f-ed4956ab96a5', '535386bd-b0e0-541a-ba30-c4579db6f8aa', 'e140379d-82dc-5d26-b182-3ff7f77f03b2'],
'65c843d5-e8b8-537f-a6c4-3ccb301d09e5':['6deb9c74-e7ed-551a-b88f-a107678b22c1', 'dad69934-838a-5abb-99cc-0661feb0b95d', 'aa0ef5d1-7a0e-518d-b93d-1834db5bae48', 'ddc78420-8f1e-5656-a2fa-c967f278aad1', 'cdd63ae9-1a94-5a88-ad44-4c68105d51fa', 'ba3be4f0-592d-5d72-a96b-5c461c871c9b', '005d3b86-14de-58a3-86ac-16ceff561782', '71b50e66-cabc-5697-8e58-2c43648932db', 'eb6a1519-3559-5797-ac35-d31505d66961', '01289fb6-5c85-50ba-a8b9-bfa754d5aeb5'],
'65418e9d-3d93-505c-94f5-df49bfa49847':['89d4bd99-1a51-568a-bd62-668c227a3595', '26e37b90-e471-5595-abed-bbe8d69e5f1e', '6720b5a6-7bb0-53fa-a8ad-96d9ab01ffe8', 'eae0a255-8263-5ce2-a9d2-a647b9d9273e', '5e906b7d-5d87-587a-a1da-8daba92d28c4'],
'280670b2-7d06-535c-80b7-4438a537a3fc':['32ad8fca-24f6-5b33-b2fa-76f1956f52ad', '1449612a-5f9b-5960-8114-2da760f63c5e', '0d2b68da-87c5-5ef2-89cf-269e00c55ad0', '158cea75-7d77-5f25-b54b-8a5c5fa808fe'],
'b478e741-bb5c-5f1e-ac5c-e58ea15f2358':['3ff2338c-3155-5eaa-a5fa-daa4eaac8b35', '4a9a10a4-45db-5bc9-b79b-715e55109195', 'cc57ea78-01eb-555a-82fd-556d731b6a00', '35aebfd6-8e1a-5cfd-a2e9-2fb75a231ce9', 'c41905d1-28e9-5510-a8ed-e547ac81a141', '236291ef-da5c-5d7e-9aa1-b05d0170d10f', '82c2627c-9d57-591b-b149-bf9c4db05162'],
'87ef73f2-887a-52b0-aa4f-2e1b8d3676cb':['7f1e68e8-3cfb-521a-aa83-aa38c8b8714f', 'a18ecb25-b898-5b03-907a-04067695f9b1', 'fdcb3a4c-1db3-597c-b885-093937c8a68c', 'c918d16d-8bf9-5ae1-a32d-7d0cf78245f0', '9f555599-a017-57f2-8e42-a3eaa845c1f9', '4c4827c7-7501-5c87-a954-b2e186b455d7', '47bd74b8-f707-5094-83dd-fe2e8f0e0dcb', '98ac98f5-d676-5eee-9602-c874f6d86f15'],
'ff46835d-4bb0-592b-9c5f-470b9bdb9d76':['4c0daf64-b423-56db-8f1b-c83344b6f683', '6614e817-c2ae-56ba-8419-5965ab8600ed', '6bfc31d2-afca-56b3-a5f4-d9f19e628c55'],
'e5e55e0c-98f2-57e6-b252-2acd32197a33':['c5cfd93b-0b81-575a-bc65-952d2d6d68fc', '4003165a-47fa-5b8b-9430-d553e3969130', '7689e398-0970-5c18-ba93-e0b14013ab30', '48fa5894-3be0-5046-b99b-96d65d13fbc7'],
'311b72c3-b538-5a62-b2c8-e340ae014465':['aec24c7c-c835-5212-ac86-09f120b762e6', 'e03cdac1-b51d-5de7-bd1f-81d8d84b6d26', '8e417d85-109c-55f2-adc9-f45557b23eaf', '1ff6f9ec-3eb9-547f-be97-4e8ec82a1bc7'],
'fb7f54de-114b-56e9-9fc5-6c3011e104ab':['e4a6a41f-4737-545b-8e8b-938be76fa906', '876fd01f-2096-5772-bd06-dc91ffc165d7', '491918a7-5259-5a5e-b01f-4842a98ec78d'],
'35615ff3-9c28-5f4d-96d4-75ee7e861ab4':['c9004e51-ac87-58aa-8404-4f0543bdab21', 'a2ec645e-4b24-5995-ac16-a91d2fe427d8', '893f7929-8f1e-5472-a209-503b6fb1b028', '3566443e-27c9-5f5b-8627-905d548c484d', '1daea858-e9e3-56ee-bf3e-5ec7a2229d31'],
'fc7226d1-910a-5010-bdb9-2d7865355965':['7b961b85-ed5e-5c58-adc4-74e2cd714399', '705b29f0-5d25-5da9-83e6-1110113eeb69', '6a99e295-db08-5598-9746-10848bd2871e', 'fd7e1ab7-311d-5f0b-ac20-35aa8050659e', 'b5702e82-21e5-5a8f-b0a2-1d349911cc15', 'd45935d0-ac75-510a-b15c-56b4845e4a9e', 'd3189f4b-7dd2-5751-a8d2-a2121b0b4790', 'c89752eb-ebd8-5c4b-b264-b8aa49b25a4c', '62988cd5-c570-5a40-afb2-978322885cbf'],
'70b69899-4663-5c84-9a18-e29f38661aea':['fc7226d1-910a-5010-bdb9-2d7865355965', '705b29f0-5d25-5da9-83e6-1110113eeb69', 'c89aa997-e62c-53c5-a3ae-e657297d208a', 'fd7e1ab7-311d-5f0b-ac20-35aa8050659e', 'ac604aa7-1cfc-56b0-976d-0faafd49e711', 'd3189f4b-7dd2-5751-a8d2-a2121b0b4790', '93dff7a6-8bec-5779-9fa7-d31aa2974689', 'c89752eb-ebd8-5c4b-b264-b8aa49b25a4c'],
}

private_labels = {'08884f97-0714-57a9-a356-fc0fb67dd4ac': ['89f421e8-0aa0-5a71-b8c2-9880b83844d9', '611e5303-4308-5924-9a8a-152af993f001', '3a011750-a6e8-511b-b899-f77a4e6d596e', '781fbc7e-9690-55c8-b179-2bb7cded8cf9'], 
                  '5b28ef54-facf-5f87-b37b-d2cee2cdd466': ['6715d0e1-a229-5de7-b6e3-046a8e103e55'], 
                  '39b70d92-2958-5ecf-abcb-a772b565afae': ['a59853ba-5513-5b03-baae-c41c82eda078', 'a8658765-5012-5bac-a052-39e1ada83bd7', '44c749dd-2f92-56b9-831d-95450655a245', 'c20fa5cc-7a19-569a-80d0-6dcc8579cb8c'], 
                  'a0b22be6-dd87-5e6a-af12-f967ed7f481d': ['b562fb43-42a9-5eea-aa30-0d6c5812dc58', '74fc7730-afff-5593-aa11-ad68af06cd36', '0df40d87-a948-5a65-9a02-0487173aa75a', '53cfd17f-fe28-51e0-8db6-a334e43496cf', 'b3227a8e-b565-5a27-9dcf-b0f3ff4c1ae5']
                 }


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
class Company(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()
    country = graphene.String()
    sector = graphene.String()
    revenue = graphene.Float()
    ebitda = graphene.Float()
    num_employees = graphene.Int()
    score = graphene.Float()
    region = graphene.String()
    description = graphene.String()
    company_type = graphene.String()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
target_ids = list(public_labels.keys())
target_ids += sum([v for _,v in public_labels.items()], [])

In [13]:
companies_raw = (spark.read.load("s3://ai-data-lake-dev-eu-west-1/business/company_data_denormalized")
                 .filter(F.col("primary_sic_node_desc").isNotNull() &
                         (F.length("primary_sic_node_desc") > 0)
                        )
                 .join(spark.read.load(data_path),
                       on = ["entity_id"]
                      )
                 .select("entity_id",
                         "business_description",
                         "bow_description_lemmatized", 
                         "primary_sic_node_desc", 
                         "total_revenue",
                         "name",
                         "region",
                         "ebitda",
                         "country_of_incorporation",
                         "company_type_name",
                         "number_employees"
                        )
                 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def enhance_description(description, industry, mu=10.0, topn=30000):
    total = sum(ct for _,ct in description.items())
    document_dist = {word:ct for word,ct in description.items()}

    total_p = 0.0
    if industry in industry2idx:
        for ind,val in islice(sorted(enumerate(phi[industry2idx[industry]]), key=lambda x: x[1], reverse=True), topn):
            word = id2word[ind]
            new_val = document_dist.get(word, 0.0) + mu*val
            document_dist[word] = new_val
            total_p += val

    return {word:float(ct/(total+total_p*mu)) for word,ct in document_dist.items()}

enhance_description_udf = F.udf(enhance_description, T.MapType(T.StringType(), T.FloatType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
def kl_distance_udf(target):
    def kl_distance(peer):
        ret = 0
        for word,prob in target.items():
            if prob == 0.0:
                continue
            ret += prob*math.log(prob/(peer.get(word,0.0)+1e-9))
        return ret
    return F.udf(kl_distance, T.FloatType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
companies_smooth = companies_raw.select(
    "entity_id", 
    "bow_description_lemmatized",
    "business_description",
    "primary_sic_node_desc", 
    "total_revenue",
    "name",
    "region",
    "ebitda",
    "country_of_incorporation",
    "company_type_name",
    "number_employees",
    enhance_description_udf("bow_description_lemmatized", "primary_sic_node_desc").alias("description_smooth")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
companies_smooth.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

866497

In [18]:
def make_company_from_rdd(hit):
    id = hit.entity_id
    score = hit.dist
    cname = hit.name
    sector = (
        hit.primary_sic_node_desc
        if hit.primary_sic_node_desc is not None
        else ""
    )
    country = hit.country_of_incorporation if hasattr(hit, "country_of_incorporation") else ""
    company_type = hit.company_type_name if hasattr(hit, "company_type_name") else ""

    revenue = hit.total_revenue if hit.total_revenue is not None else 0
    num_employees = (
        hit.number_employees if hit.number_employees is not None else 0
    )

    region = hit.region if hit.region is not None else "n/a"

    description = hit.business_description if hit.business_description is not None else ""

    ebitda = hit.ebitda if hit.ebitda is not None else 0

    return Company(
                id=id,
                name=cname,
                revenue=revenue,
                ebitda=ebitda,
                sector=sector,
                num_employees=num_employees,
                country=country,
                score=score,
                region=region,
                description=description,
                company_type=company_type,
            )

def obtain_result(companies_smooth, target_id, max_size=100, gp_revenue_range=False):
    target_doc = (companies_smooth.filter(F.col("entity_id") ==  target_id)
                  .select("description_smooth")
                  .first()
                 )
    if target_doc is None:
        print(f"Failed to find {target_id}.")
        return []
    target_doc = target_doc[0]
    target_revenue = (companies_smooth.filter(F.col("entity_id") ==  target_id)
                      .select("total_revenue")
                      .first()[0]
                     )
    if not gp_revenue_range:
        revenue_min = target_revenue*0.1
        revenue_max = target_revenue/0.1
    else:
        transformed_revenue = x_scale.transform(np.array([[target_revenue]]))
        _, predicted_std = m.predict(transformed_revenue)
        
        revenue_min = (target_revenue - 0.1*predicted_std)[0,0]
        revenue_max = (target_revenue + 1.96*predicted_std)[0,0]
    
    res = (companies_smooth
     .filter((F.col("total_revenue") >= revenue_min) &
             (F.col("total_revenue") <= revenue_max)
            )
     .select("entity_id",
             "business_description",
             "primary_sic_node_desc", 
             "total_revenue",
             "name",
             "region",
             "ebitda",
             "country_of_incorporation",
             "company_type_name",
             "number_employees",
             kl_distance_udf(target_doc)("description_smooth").alias("dist"))
     .orderBy("dist")
    ).take(max_size)
    
    return [
        make_company_from_rdd(el) for el in res
    ]

def find_peer_in_result(results, peer):
    for i, company in enumerate(results):
        if peer == company.id:
            return i+1

    return -1

def add_ranks_in_tops(ranks, counts):
    for rank in ranks:
        if 0 < rank <= 10:
            counts[0] += 1
        elif 10 < rank <= 20:
            counts[1] += 1
        elif 20 < rank <= 50:
            counts[2] += 1
        elif 50 < rank <= 100:
            counts[3] += 1
        elif 100 < rank:
            counts[4] += 1
        else:
            counts[5] += 1
    return counts

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Manual revenue filtering, lemmatized

In [19]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers = defaultdict(list)
suggested_peers = {}
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(companies_smooth, query_id)
    ranks = []
    
    suggested_peers[public_label] = results
    
#     for peer in results[:4]:
#         missed_peer = client.get(index='company', id=peer.id)

#         revenue = missed_peer['_source'].get('total_revenue', 0)
#         description = missed_peer['_source'].get('business_description', 0)
#         name = missed_peer['_source'].get('name', 0)
#         print("Found peer")
#         print(name)
#         print(description)
#         print(revenue)

    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)
#             missed_peer = client.get(index='company', id=peer)

#             revenue = missed_peer['_source'].get('total_revenue', 0)
#             description = missed_peer['_source'].get('business_description', 0)
#             name = missed_peer['_source'].get('name', 0)
#             print("Missed peer")
#             print(name)
#             print(description)
#             print(revenue)


    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)

    print(ranks)
#     print("=\n"*10)

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['1b3872ba-8913-517f-a253-7e62a525cf55', 21, 5, 13, 2]
['7689e398-0970-5c18-ba93-e0b14013ab30', -1, -1, -1]
['cb997a23-552a-52b9-acf5-bcdce2d1eea1', -1, -1, -1]
['e62877cd-15cd-561d-a77e-8a5bccf9873c', 8, 6]
['29f2b914-f655-5a0e-8642-252840119f94', 14, 39, -1]
['c655ffcb-391e-57af-9d57-28586381be92', 2, 7, -1]
['b01d22f1-2b08-5a1b-937e-bd29be833832', -1, -1, -1, -1, 11, -1, -1, -1, -1, -1]
['6a99e295-db08-5598-9746-10848bd2871e', 14, 4, 2]
['eb6a1519-3559-5797-ac35-d31505d66961', 2, -1, 54, -1, 3, 7, -1]
['94a8555e-9f82-5657-9f32-f44c1844e371', 14, -1, 20, 2, 29]
['4dfb3296-c5ab-5d27-bcaa-71a67990dc0f', -1, -1]
['3e71b765-5a7c-54b0-8687-fbb479e5ce91', 69, 13, 99, -1, -1, -1]
['4b90ea7b-dd01-5114-b2b1-e44f8b0f496c', -1, 27, -1, 31, -1]
['0eac306f-7b21-534f-b7eb-2fe3dbf0dcb9', 15, -1, -1, -1, -1, -1]
['6d6a0abb-b4fa-5dfa-b44e-35927bff8864', -1, 3, -1, -1, -1, 88]
['3f02111a-3779-53cb-b556-ef2b02712eb8', -1, 4, -1, -1, -1]
['a2ec645e-4b24-5995-ac16-a91d2fe427d8', -1, 35, 28, 23, 4, 12, 29

In [20]:
def company_to_dict(c):
    return {
        "id": c.id,
        "name": c.name,
        "country": c.country,
        "sector": c.sector,
        "revenue": c.revenue,
        "ebitda": c.ebitda,
        "num_employees": c.num_employees,
        "score": c.score,
        "region": c.region,
        "description": c.description,
        "company_type": c.company_type,
    }

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
with smart_open.open(f"{model_path}/suggested_peers.pkl", "wb") as f:
    pickle.dump({k:[company_to_dict(el) for el in v] for k,v in suggested_peers.items()}, f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…